In [7]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [8]:
N_e = 10000
modes = ["constant", "recomb_map"]
panel_labels = ["A", "B"]

LABEL_FONTSIZE = 24
TICK_FONTSIZE  = 16
LEGEND_FONTSIZE = 14

sns.set_theme(style="white", palette="colorblind")
palette = sns.color_palette("colorblind")

In [9]:
def load_observed(N_e, mode):
    if mode == "constant":
        obs_path = f"obs_switches/constant_recomb/summary_obs_switches_n_{N_e}.csv"
    elif mode == "recomb_map":
        obs_path = f"obs_switches/recomb_map/summary_obs_switches_n_{N_e}.csv"
    else:
        raise ValueError("mode must be 'constant' or 'recomb_map'")

    if not os.path.exists(obs_path):
        raise FileNotFoundError(f"No observed summary file found: {obs_path}")
    return pd.read_csv(obs_path)

In [10]:
def load_expected(N_e, mode):
    if mode == "constant":
        exp_path = f"exp_switches/constant_recomb/expected_switches_{N_e}.csv"
    elif mode == "recomb_map":
        exp_path = f"exp_switches/recomb_map/expected_switches_{N_e}.csv"
    else:
        raise ValueError("mode must be 'constant' or 'recomb_map'")

    if not os.path.exists(exp_path):
        raise FileNotFoundError(f"No expected file found: {exp_path}")
    return pd.read_csv(exp_path)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 6), sharey=False)

for ax, mode, panel in zip(axes, modes, panel_labels):
    # Load data
    df_obs = load_observed(N_e, mode)
    df_exp = load_expected(N_e, mode)

    # Plot observed mean + CI
    ax.plot(df_obs["generation"], df_obs["obs_mean"],
            marker="o", linewidth=3, label="Simulated", color=palette[0])
    ax.fill_between(df_obs["generation"], df_obs["obs_ci_lower"], df_obs["obs_ci_upper"],
                    alpha=0.3, label="Simulated 95% CI", color=palette[0])

    # Plot expected
    ax.plot(df_exp["generation"], df_exp["expected_switches"],
            linestyle="--", linewidth=3, label="Theoretical", color=palette[1])

    # Tick/axis formatting
    ax.tick_params(axis="both", which="major",
                   direction="out", length=6, width=1.5,
                   bottom=True, left=True, labelsize=TICK_FONTSIZE)
    for spine in ["left", "bottom"]:
        ax.spines[spine].set_visible(True)
        ax.spines[spine].set_linewidth(1.2)

    ax.legend(fontsize=LEGEND_FONTSIZE)

    # Panel letter
    ax.text(-0.1, 1.05, panel, transform=ax.transAxes,
            fontsize=LABEL_FONTSIZE, fontweight="bold", va="top", ha="right")

    # Mode label
    label_txt = "Constant recombination" if mode == "constant" else "Recombination map"
    ax.text(0.95, 0.05, label_txt, transform=ax.transAxes,
            fontsize=LABEL_FONTSIZE-2, ha="right", va="bottom", fontstyle="italic")

axes[0].set_ylim(0, 12)
axes[1].set_ylim(0, 40)

# Generate proportional ticks so the grid lines match visually
left_ticks = np.linspace(0, 12, 3)           
right_ticks = np.linspace(0, 40, 5)          

axes[0].set_yticks(left_ticks)
axes[1].set_yticks(right_ticks)

# Optional rounding for neat labels
axes[0].set_yticklabels([f"{t:.0f}" if t.is_integer() else f"{t:.1f}" for t in left_ticks])
axes[1].set_yticklabels([f"{int(t)}" for t in right_ticks])